In [1]:
!pip install wandb -q

In [2]:
# Getting wandb api key in secrets

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")

In [3]:
import wandb
wandb.login(key=secret_value_0)

print(wandb.__version__)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m007 (da24m007-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


0.19.6


In [4]:
import torchvision.transforms as T

def get_transforms():
    IMAGE_SIZE = (224, 224) 
    return T.Compose([
        T.Resize(IMAGE_SIZE),
        T.ToTensor(),
        T.Normalize(mean=[0.5]*3, std=[0.5]*3)  # normalize to [-1, 1]
    ])

In [5]:
# importing libraries

import os
import shutil
from sklearn.model_selection import train_test_split

In [6]:
data_dir = '/kaggle/input/nature-12k-new/inaturalist_12K/train'
new_val_dir = 'val_for_tuning'
new_train_dir = 'train_for_tuning'
os.makedirs(new_val_dir, exist_ok=True)
os.makedirs(new_train_dir, exist_ok=True)

In [7]:
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir,class_name)

    # error handling in case the class_path does not exist
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    train_imgs, val_imgs = train_test_split(images,test_size=0.2, random_state=42)

    # create a new directory for storing val_imgs
    os.makedirs(os.path.join(new_val_dir, class_name), exist_ok = True)
    os.makedirs(os.path.join(new_train_dir, class_name), exist_ok = True)

    for img in val_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(new_val_dir, class_name, img))

    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(new_train_dir, class_name, img))

In [8]:
from torch.utils.data import DataLoader
from torchvision import datasets

# Datasets
train_dataset = datasets.ImageFolder(root=new_train_dir, transform=get_transforms())
val_dataset = datasets.ImageFolder(root=new_val_dir, transform=get_transforms())
test_dataset = datasets.ImageFolder(root='/kaggle/input/nature-12k-new/inaturalist_12K/val', transform=get_transforms())

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [9]:
wandb.init(project="DL-A2-V2", name="resnet50-finetune", config={
    "epochs": 10,
    "batch_size": 32,
    "lr_fc": 1e-3,
    "lr_base": 1e-4,
    "model": "resnet50",
    "strategy": "unfreeze-last-2-blocks"
})


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze last two blocks
for name, child in list(model.named_children())[-3:]:  
    for param in child.parameters():
        param.requires_grad = True

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 180MB/s] 


In [11]:
# Use different learning rates
optimizer = optim.Adam([
    {'params': model.layer3.parameters(), 'lr': wandb.config.lr_base},
    {'params': model.layer4.parameters(), 'lr': wandb.config.lr_base},
    {'params': model.fc.parameters(), 'lr': wandb.config.lr_fc},
])

criterion = nn.CrossEntropyLoss()


In [12]:
for epoch in range(wandb.config.epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Optionally validate
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)
    
    val_loss = val_loss / val_total
    val_acc = val_correct / val_total

    # 🚀 Log to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

    print(f"[Epoch {epoch+1}] Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")


[Epoch 1] Train Acc: 0.6893, Val Acc: 0.7650
[Epoch 2] Train Acc: 0.8545, Val Acc: 0.7540
[Epoch 3] Train Acc: 0.9247, Val Acc: 0.7610
[Epoch 4] Train Acc: 0.9484, Val Acc: 0.7640
[Epoch 5] Train Acc: 0.9614, Val Acc: 0.7770
[Epoch 6] Train Acc: 0.9690, Val Acc: 0.7585
[Epoch 7] Train Acc: 0.9801, Val Acc: 0.7765
[Epoch 8] Train Acc: 0.9734, Val Acc: 0.7680
[Epoch 9] Train Acc: 0.9761, Val Acc: 0.7685
[Epoch 10] Train Acc: 0.9785, Val Acc: 0.7660
